In [1]:
import roboflow
import os
from ultralytics import YOLO

import gdown

In [11]:
ROBOFLOW_API_KEY = os.getenv("ROBOFLOW_API_KEY")

if ROBOFLOW_API_KEY is None:
    raise ValueError("ROBOFLOW_API_KEY environment variable is not set.")
    
rf = roboflow.Roboflow(api_key=ROBOFLOW_API_KEY)
project = rf.workspace("meus-projetos-rzl4g").project("road-signs-8").version(1)
dataset = project.download("yolov11", location="data/road_signs")

print(f"Dataset downloaded to: {dataset.location}")

loading Roboflow workspace...
loading Roboflow project...
Exporting format yolov11 in progress : 85.0%
Version export complete for yolov11 format



Extracting Dataset Version Zip to data/road_signs in yolov11:: 100%|██████████| 1612/1612 [00:00<00:00, 14308.61it/s]

Dataset downloaded to: /home/pauloricms/Documents/stb/desafio/data/road_signs


In [6]:
gdown.download(id = '1q1yRrapzg5N4f3kLEGF0xWqSD5Qc0atX', output='data/traffic_videos/clips.mp4', quiet=False)

Downloading...
From: https://drive.google.com/uc?id=1q1yRrapzg5N4f3kLEGF0xWqSD5Qc0atX
To: /home/pauloricms/Documents/stb/desafio/data/traffic_videos/clips.mp4
100%|██████████| 43.1M/43.1M [00:01<00:00, 33.6MB/s]


'data/traffic_videos/clips.mp4'

In [2]:
model = YOLO("yolo11n.pt")

results = model.train(
    data="/home/pauloricms/Documents/stb/desafio/data/road_signs/data.yaml",
    epochs=200,      
    imgsz=640,
    batch=12,       
    device=0,
    project="road_signs_project",
    name="yolo11n_run"
)

Ultralytics 8.3.235 🚀 Python-3.12.3 torch-2.9.1+cu128 CUDA:0 (NVIDIA GeForce GTX 1650, 3716MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=12, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/home/pauloricms/Documents/stb/desafio/data/road_signs/data.yaml, degrees=0.0, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=200, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolo11n.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=yolo11n_run, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, overlap_mask=Tr

In [3]:
model = YOLO('road_signs_project/yolo11n_run/weights/best.pt')

# Export the model to TensorRT engine format with batch size 32, dynamic shapes, half(FP16) precision, on device 0
model.export(format='engine',
             batch=16,
             workspace=4,
             half=True, 
             device=0)

Ultralytics 8.3.235 🚀 Python-3.12.3 torch-2.9.1+cu128 CUDA:0 (NVIDIA GeForce GTX 1650, 3716MiB)
YOLO11n summary (fused): 100 layers, 2,583,712 parameters, 0 gradients, 6.3 GFLOPs

PyTorch: starting from 'road_signs_project/yolo11n_run/weights/best.pt' with input shape (16, 3, 640, 640) BCHW and output shape(s) (16, 12, 8400) (5.2 MB)

ONNX: starting export with onnx 1.19.1 opset 20...
ONNX: slimming with onnxslim 0.1.78...
ONNX: export success ✅ 0.9s, saved as 'road_signs_project/yolo11n_run/weights/best.onnx' (10.1 MB)

TensorRT: starting export with TensorRT 10.14.1.48.post1...
[12/06/2025-17:01:36] [TRT] [I] ----------------------------------------------------------------
[12/06/2025-17:01:36] [TRT] [I] Input filename:   road_signs_project/yolo11n_run/weights/best.onnx
[12/06/2025-17:01:36] [TRT] [I] ONNX IR version:  0.0.9
[12/06/2025-17:01:36] [TRT] [I] Opset version:    20
[12/06/2025-17:01:36] [TRT] [I] Producer name:    pytorch
[12/06/2025-17:01:36] [TRT] [I] Producer version: 

'road_signs_project/yolo11n_run/weights/best.engine'